# ~~Чихай~~ Читай

**Цели исследования:** 

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

**Описание исследования:**

Компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Задача — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.


In [ ]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Схема базы данных

![Тег изображения](https://concrete-web-bad.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?table=block&id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=1640&userId=&cache=v2)

## Знакомство с данными

In [ ]:
# выведем 5 первых строк каждой таблицы
print()
print('-' * 80)
print()
for table in ['books', 'authors', 'ratings', 'reviews', 'publishers']:
    query = (
        f'''SELECT *
            FROM {table}
            LIMIT 5
            ''')
    print(f'Первые 5 строк из таблицы {table}')
    display(pd.io.sql.read_sql(query, con = engine))
    print()
    print('-' * 80)
    print()


--------------------------------------------------------------------------------

Первые 5 строк из таблицы books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



--------------------------------------------------------------------------------

Первые 5 строк из таблицы authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



--------------------------------------------------------------------------------

Первые 5 строк из таблицы ratings


...


--------------------------------------------------------------------------------

Первые 5 строк из таблицы reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...



--------------------------------------------------------------------------------

Первые 5 строк из таблицы publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



--------------------------------------------------------------------------------



## Посчитать, сколько книг вышло после 1 января 2000 года

In [ ]:
query = (
'''SELECT COUNT(book_id) AS book_count
   FROM books
   WHERE publication_date > CAST('01/01/2000' as date)
''')

pd.io.sql.read_sql(query, con = engine) 

,book_count
0,819


В нашем приложении не так уж и много книг.

## Для каждой книги посчитайть количество обзоров и среднюю оценку

In [ ]:
query = (
'''SELECT b.book_id,
          b.title,
          q1.review_count, 
          ROUND(q2.avg_rating, 2)
   FROM books AS b
   LEFT JOIN (SELECT b.book_id,
                     COUNT(DISTINCT rv.review_id) AS review_count
              FROM books AS b
              LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
              GROUP BY b.book_id
              ) AS q1 ON q1.book_id=b.book_id
   LEFT JOIN (SELECT b.book_id,
                     AVG(r.rating) AS avg_rating
              FROM books AS b
              LEFT JOIN ratings AS r ON b.book_id = r.book_id
              GROUP BY b.book_id
              ) AS q2 ON q2.book_id=b.book_id
  ORDER BY 3 desc, q2.avg_rating DESC
   
''')

pd.io.sql.read_sql(query, con = engine) 

,book_id,title,review_count,round
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Посчитали.

## Определить издательство, которое выпустило наибольшее число книг толще 50 страниц
(так мы исключим из анализа брошюры)

In [ ]:
query = (
'''SELECT p.publisher,
          COUNT(b.book_id) AS books_cnt
   FROM books AS b
   LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
   WHERE b.num_pages > 50
   GROUP BY p.publisher
   ORDER BY COUNT(b.book_id) DESC
   LIMIT 1
''')

pd.io.sql.read_sql(query, con = engine) 

,publisher,books_cnt
0,Penguin Books,42


Penguin Books — британское издательство, основанное в 1935 году в Лондоне сэром Алленом Лейном и его братьями Ричардом и Джоном. Главной заслугой издательства считают демократизацию книжного рынка, превратившую книгу из предмета роскоши в удовольствие, доступное массам. (Википедия)

## Определить автора с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками)

In [ ]:
query = (
'''SELECT a.author,
          ROUND(AVG(r.rating), 2) AS avg_rating
   FROM books AS b
   LEFT JOIN authors AS a ON a.author_id = b.author_id
   LEFT JOIN ratings AS r ON b.book_id = r.book_id
   WHERE b.book_id IN (SELECT b.book_id
                       FROM books AS b
                       LEFT JOIN ratings AS r ON b.book_id = r.book_id
                       GROUP BY b.book_id
                       HAVING COUNT(r.rating_id) >= 50)
   GROUP BY a.author
   ORDER BY AVG(r.rating) DESC
   LIMIT 1
''')

pd.io.sql.read_sql(query, con = engine) 

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


Джонана Роулинг - неудивительно))

Гарри Поттера любят все!


![Тег изображения](https://media.giphy.com/media/jNcu2YunWZBcI/giphy.gif)

# Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [ ]:
query = (
'''SELECT ROUND(AVG(q.review_count), 2) AS avg_review_count
   FROM (
        SELECT review.username,
               COUNT(review.review_id)  AS review_count
        FROM reviews AS review
        WHERE review.username IN (SELECT r.username
                                  FROM ratings AS r
                                  GROUP BY r.username
                                  HAVING COUNT(r.rating_id) > 50
                                  ) 
        GROUP BY review.username
      ) AS q
''')

pd.io.sql.read_sql(query, con = engine) 

,avg_review_count
0,24.33


Пользователи в пандемию на удивление много читают и оставляют отзывов...

# Выводы

- Приложение, видимо, очень новое, поэтому ассортимент книг довольно скудный.  
   Книг, выпущенных после 2000 года в приложении всего `819`.
   
   
- Пока что наибольшее количество обзоров на одну книгу - 7, не так уж и много. Но и немало. 
  Приложение новое, но пользователи ведут себя достаточно активно.
  
  
- Издательство, выпустившее наибольшее количество книг длиннее 50 страниц - Penguin Books — британское издательство, основанное в 1935 году в Лондоне сэром Алленом Лейном и его братьями Ричардом и Джоном. Главной заслугой издательства считают демократизацию книжного рынка, превратившую книгу из предмета роскоши в удовольствие, доступное массам. 

   Возможно, с ними можно начать сотрудничать, чтобы расширить ассортимент приложения.
   
- Самая высокая средняя оценка книг в нашем приложении достается автору саги о Гарри Поттере - Джоане Роулинг. Это совсем неудивительно, так как эти книги знает и любит практически весь земной шар. Первая книга о Гарри Поттере - `Гарри Поттер и философский камень` - одна из самых продаваемых книг в мире за все время. 


- Среднее количество обзоров от людей, поставивших более 50 оценкам - около 24. Пользователи этого приложения действительо ведут себя очень активно. Вероятно, приложение им нравится. 

Если расширить асссортимент книг - все станет еще лучше!